In [1]:
!pip install psycopg2

In [ ]:
import requests

url = "https://realty-mole-property-api.p.rapidapi.com/properties"

querystring = {"address":"5500 Grand Lake Dr, San Antonio, TX, 78244"}

headers = {
	"x-rapidapi-key": "3e7f4fa5ecmsh410c7d813c60e5cp1dbf74jsn09e7c2369927",
	"x-rapidapi-host": "realty-mole-property-api.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

print(response.json())

In [ ]:
import requests

url = "https://realty-mole-property-api.p.rapidapi.com/randomProperties"

querystring = {"limit":"100000"}

headers = {
	"x-rapidapi-key": "3e7f4fa5ecmsh410c7d813c60e5cp1dbf74jsn09e7c2369927",
	"x-rapidapi-host": "realty-mole-property-api.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

#print(response.json())

In [ ]:
data = response.json
#save the data to a file
filename = 'PropertyRecords.json'
with open(filename, 'w') as file:
    json.dump(data, file, indent=4)

In [1]:
import requests
import json
import pandas as pd
import csv
import psycopg2
from psycopg2 import sql

# extraction layer

In [2]:
# read data into a df
#property_record = pd.read_json('PropertyRecords.json')
property_record= pd.read_json(r"C:\Users\HP\Downloads\dataset-20240731T151644Z-001\dataset\PropertyRecords.json")

In [3]:
property_record.head()

,addressLine1,city,state,zipCode,formattedAddress,assessorID,county,legalDescription,ownerOccupied,squareFootage,...,bathrooms,features,taxAssessment,propertyTaxes,owner,id,longitude,latitude,bedrooms,addressLine2
0,30631 W Celeborn Dr,Buckeye,AZ,85396,"30631 W Celeborn Dr, Buckeye, AZ 85396",504-75-191,Maricopa,TARTESSO UNIT 2A MCR 754-28,1.0,2025.0,...,3.0,"{'exteriorType': 'Stucco', 'floorCount': 1, 'g...","{'2022': {'value': 2420, 'land': 2420}, '2023'...","{'2021': {'total': 125}, '2022': {'total': 1593}}","{'names': ['ADRIAN RAMIREZ', 'MONICA JOANNA LO...","30631-W-Celeborn-Dr,-Buckeye,-AZ-85396",-112.710832,33.481631,NaN,NaN
1,1212 Horseman Pl,Bismarck,ND,58501,"1212 Horseman Pl, Bismarck, ND 58501",1389-003-035,Burleigh,SLEEPY HOLLOW HEIGHTS 5TH LOT 8,1.0,2306.0,...,3.0,"{'architectureType': 'French Provincial', 'coo...","{'2022': {'value': 184550, 'land': 37000, 'imp...",{'2022': {'total': 5158}},"{'names': ['ZACHARY P BOEHLER'], 'mailingAddre...","1212-Horseman-Pl,-Bismarck,-ND-58501",-100.746177,46.818761,4.0,NaN
2,11030 Creekbridge Pl,San Diego,CA,92128,"11030 Creekbridge Pl, San Diego, CA 92128",316-232-07-09,San Diego,PM15983 PAR 2*US 74PER DOC90-202308&UND INT IN,NaN,1273.0,...,2.5,"{'architectureType': 'Condo / Apartment', 'gar...","{'2020': {'value': 329281, 'land': 151552, 'im...","{'2020': {'total': 3559}, '2022': {'total': 36...","{'names': ['INGRID E ENSCHSIMON'], 'mailingAdd...","11030-Creekbridge-Pl,-San-Diego,-CA-92128",-117.092509,32.945955,2.0,NaN
3,1113 W High St,Haddon Heights,NJ,8035,"1113 W High St, Haddon Heights, NJ 08035",NaN,Camden County,NaN,NaN,1936.0,...,1.5,NaN,NaN,NaN,NaN,"1113-W-High-St,-Haddon-Heights,-NJ-08035",-75.067199,39.875404,3.0,NaN
4,441 Janice Kay Pl,Kissimmee,FL,34744,"441 Janice Kay Pl, Kissimmee, FL 34744",292530299600010780,Osceola,EAGLES LANDING PB 15 PGS 121-122 LOT 78,1.0,2620.0,...,2.0,"{'cooling': True, 'coolingType': 'Package', 'e...",{'2022': {'value': 136633}},{'2022': {'total': 2823}},"{'names': ['MEADEMATIAS JESSICA', 'MATIAS REIN...","441-Janice-Kay-Pl,-Kissimmee,-FL-34744",-81.335555,28.287159,3.0,NaN


In [4]:
property_record.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 27 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   addressLine1      500 non-null    object 
 1   city              500 non-null    object 
 2   state             500 non-null    object 
 3   zipCode           500 non-null    int64  
 4   formattedAddress  500 non-null    object 
 5   assessorID        338 non-null    object 
 6   county            500 non-null    object 
 7   legalDescription  327 non-null    object 
 8   ownerOccupied     255 non-null    float64
 9   squareFootage     389 non-null    float64
 10  subdivision       289 non-null    object 
 11  yearBuilt         343 non-null    float64
 12  zoning            179 non-null    object 
 13  lotSize           330 non-null    float64
 14  propertyType      418 non-null    object 
 15  lastSalePrice     229 non-null    float64
 16  lastSaleDate      289 non-null    object 
 1

In [5]:
property_record.columns

Index(['addressLine1', 'city', 'state', 'zipCode', 'formattedAddress',
       'assessorID', 'county', 'legalDescription', 'ownerOccupied',
       'squareFootage', 'subdivision', 'yearBuilt', 'zoning', 'lotSize',
       'propertyType', 'lastSalePrice', 'lastSaleDate', 'bathrooms',
       'features', 'taxAssessment', 'propertyTaxes', 'owner', 'id',
       'longitude', 'latitude', 'bedrooms', 'addressLine2'],
      dtype='object')

# Transformation Layer

In [6]:
# Check the first element in the 'features' column
property_record.iloc[0]['features']

{'exteriorType': 'Stucco',
 'floorCount': 1,
 'garage': True,
 'garageSpaces': 2,
 'garageType': 'Garage',
 'heating': True}

To work with the features column, which contains JSON-like strings, you need to:
Parse the JSON Strings: Convert the JSON strings into Python dictionaries.
Normalize and Expand: Expand these dictionaries into separate columns in your DataFrame.

In [7]:
# convert any column(dictionary) with nested parameter/interior
#Either to convert the columns as string
# or
# Flatten it out and expand the table


In [8]:
### convert as string
# 1st step
#property_record['features'] = property_record['features'].apply(json.dumps)

In [9]:
# Check the type of the first element in the 'features' column
print(type(property_record.iloc[0]['features']))

<class 'dict'>


In [10]:
property_record.head()

,addressLine1,city,state,zipCode,formattedAddress,assessorID,county,legalDescription,ownerOccupied,squareFootage,...,bathrooms,features,taxAssessment,propertyTaxes,owner,id,longitude,latitude,bedrooms,addressLine2
0,30631 W Celeborn Dr,Buckeye,AZ,85396,"30631 W Celeborn Dr, Buckeye, AZ 85396",504-75-191,Maricopa,TARTESSO UNIT 2A MCR 754-28,1.0,2025.0,...,3.0,"{'exteriorType': 'Stucco', 'floorCount': 1, 'g...","{'2022': {'value': 2420, 'land': 2420}, '2023'...","{'2021': {'total': 125}, '2022': {'total': 1593}}","{'names': ['ADRIAN RAMIREZ', 'MONICA JOANNA LO...","30631-W-Celeborn-Dr,-Buckeye,-AZ-85396",-112.710832,33.481631,NaN,NaN
1,1212 Horseman Pl,Bismarck,ND,58501,"1212 Horseman Pl, Bismarck, ND 58501",1389-003-035,Burleigh,SLEEPY HOLLOW HEIGHTS 5TH LOT 8,1.0,2306.0,...,3.0,"{'architectureType': 'French Provincial', 'coo...","{'2022': {'value': 184550, 'land': 37000, 'imp...",{'2022': {'total': 5158}},"{'names': ['ZACHARY P BOEHLER'], 'mailingAddre...","1212-Horseman-Pl,-Bismarck,-ND-58501",-100.746177,46.818761,4.0,NaN
2,11030 Creekbridge Pl,San Diego,CA,92128,"11030 Creekbridge Pl, San Diego, CA 92128",316-232-07-09,San Diego,PM15983 PAR 2*US 74PER DOC90-202308&UND INT IN,NaN,1273.0,...,2.5,"{'architectureType': 'Condo / Apartment', 'gar...","{'2020': {'value': 329281, 'land': 151552, 'im...","{'2020': {'total': 3559}, '2022': {'total': 36...","{'names': ['INGRID E ENSCHSIMON'], 'mailingAdd...","11030-Creekbridge-Pl,-San-Diego,-CA-92128",-117.092509,32.945955,2.0,NaN
3,1113 W High St,Haddon Heights,NJ,8035,"1113 W High St, Haddon Heights, NJ 08035",NaN,Camden County,NaN,NaN,1936.0,...,1.5,NaN,NaN,NaN,NaN,"1113-W-High-St,-Haddon-Heights,-NJ-08035",-75.067199,39.875404,3.0,NaN
4,441 Janice Kay Pl,Kissimmee,FL,34744,"441 Janice Kay Pl, Kissimmee, FL 34744",292530299600010780,Osceola,EAGLES LANDING PB 15 PGS 121-122 LOT 78,1.0,2620.0,...,2.0,"{'cooling': True, 'coolingType': 'Package', 'e...",{'2022': {'value': 136633}},{'2022': {'total': 2823}},"{'names': ['MEADEMATIAS JESSICA', 'MATIAS REIN...","441-Janice-Kay-Pl,-Kissimmee,-FL-34744",-81.335555,28.287159,3.0,NaN


In [11]:
property_record.isnull().sum()

addressLine1          0
city                  0
state                 0
zipCode               0
formattedAddress      0
assessorID          162
county                0
legalDescription    173
ownerOccupied       245
squareFootage       111
subdivision         211
yearBuilt           157
zoning              321
lotSize             170
propertyType         82
lastSalePrice       271
lastSaleDate        211
bathrooms           126
features             72
taxAssessment       166
propertyTaxes       172
owner               166
id                    0
longitude             0
latitude              0
bedrooms            162
addressLine2        402
dtype: int64

In [12]:
property_record.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 27 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   addressLine1      500 non-null    object 
 1   city              500 non-null    object 
 2   state             500 non-null    object 
 3   zipCode           500 non-null    int64  
 4   formattedAddress  500 non-null    object 
 5   assessorID        338 non-null    object 
 6   county            500 non-null    object 
 7   legalDescription  327 non-null    object 
 8   ownerOccupied     255 non-null    float64
 9   squareFootage     389 non-null    float64
 10  subdivision       289 non-null    object 
 11  yearBuilt         343 non-null    float64
 12  zoning            179 non-null    object 
 13  lotSize           330 non-null    float64
 14  propertyType      418 non-null    object 
 15  lastSalePrice     229 non-null    float64
 16  lastSaleDate      289 non-null    object 
 1

In [18]:
# Impute missing values
property_record['ownerOccupied'].fillna('Unknown', inplace=True)
property_record['legalDescription'].fillna('Unknown', inplace=True)
property_record['subdivision'].fillna('Unknown', inplace=True)
property_record['zoning'].fillna('Unknown', inplace=True)
property_record['propertyType'].fillna('Unknown', inplace=True)
property_record['lastSaleDate'].fillna('Unknown', inplace=True)
property_record['taxAssessment'].fillna('Unknown', inplace=True)
property_record['propertyTaxes'].fillna('Unknown', inplace=True)
property_record['owner'].fillna('Unknown', inplace=True)
property_record['addressLine2'].fillna('Unknown', inplace=True)
property_record['features'].fillna('Unknown', inplace=True)

# Impute numerical columns with median
numerical_cols = [ 'bedrooms', 'bathrooms', 'squareFootage', 'lotSize']
for col in numerical_cols:
    property_record[col].fillna(property_record[col].median(), inplace=True)
    
property_record['lastSalePrice'].fillna(property_record['lastSalePrice'].mean(), inplace=True)
property_record['yearBuilt'].fillna(property_record['yearBuilt'].mode()[0], inplace=True)


# Convert 'features' column to JSON strings if necessary
def convert_to_json_string(value):
    if isinstance(value, dict):
        return json.dumps(value)
    return value

property_record['features'] = property_record['features'].apply(convert_to_json_string)

# Verify and adjust data types
property_record['lastSaleDate'] = pd.to_datetime(property_record['lastSaleDate'], errors='coerce')


In [16]:
property_record['lastSaleDate'].isnull().sum()

0

In [19]:
#option 1:I will fill missing lastSaleDate values with a placeholder date or the current date.
#This method is useful to retain a date for all records but need to clearly indicate that the date was not originally available.
#Option2: Date Interpolation
#If dates are missing in a time series context, interpolation might be useful:
# Option 1: Fill missing dates with a placeholder date
placeholder_date = pd.Timestamp('1900-01-01')
property_record['lastSaleDate'].fillna(placeholder_date, inplace=True)

# Example of forward filling, which might be useful in a time series context
#property_record['lastSaleDate'].fillna(method='ffill', inplace=True)


# handle missing values
#by filling with defaults
# drop columns/ rows as necessary

property_record.fillna({
    'assessorID': 'Unknown',
    'legalDescription': 'Not available',
    'squareFootage': 0,
    'subdivision': 'Not available',
    'yearBuilt': 0,
    'bathrooms': 0,
    'lotSize': 0,
    'propertyType': 'Unknown',
    'lastSalePrice': 0,
    'lastSaleDate':'Not available',
    'features': 'None',
    'taxAssessment': 'Not available',
    'owner': 'Unknown',
    'propertyTaxes': 'Not available',
    'bedrooms': 0,
    'ownerOccupied': 0,
    'zoning': 'Unknown',
    'addressLine2': 'Not available',
    'formattedAddress': 'Not available',
    'county': 'Not available'
    
    
}, inplace= True)

In [23]:
property_record.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 27 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   addressLine1      500 non-null    object 
 1   city              500 non-null    object 
 2   state             500 non-null    object 
 3   zipCode           500 non-null    int64  
 4   formattedAddress  500 non-null    object 
 5   assessorID        338 non-null    object 
 6   county            500 non-null    object 
 7   legalDescription  500 non-null    object 
 8   ownerOccupied     500 non-null    object 
 9   squareFootage     500 non-null    float64
 10  subdivision       500 non-null    object 
 11  yearBuilt         500 non-null    float64
 12  zoning            500 non-null    object 
 13  lotSize           500 non-null    float64
 14  propertyType      500 non-null    object 
 15  lastSalePrice     500 non-null    float64
 16  lastSaleDate      500 non-null    object 
 1

In [24]:
# facts table
fact_columns= ['addressLine1', 'city', 'state', 'zipCode', 'formattedAddress',
       'squareFootage', 'yearBuilt', 'bathrooms', 'bedrooms', 'lotSize',
       'propertyType', 'longitude', 'latitude']
fact_table = property_record[fact_columns]

In [25]:
# dimension table
# location dimension
location_dim = property_record[['addressLine1', 'city', 'state', 'zipCode', 'county', 'longitude', 'latitude']].drop_duplicates().reset_index(drop=True)
location_dim.index.name= 'location_id'

In [26]:
location_dim.head()

,addressLine1,city,state,zipCode,county,longitude,latitude
location_id,,,,,,,
0,30631 W Celeborn Dr,Buckeye,AZ,85396,Maricopa,-112.710832,33.481631
1,1212 Horseman Pl,Bismarck,ND,58501,Burleigh,-100.746177,46.818761
2,11030 Creekbridge Pl,San Diego,CA,92128,San Diego,-117.092509,32.945955
3,1113 W High St,Haddon Heights,NJ,8035,Camden County,-75.067199,39.875404
4,441 Janice Kay Pl,Kissimmee,FL,34744,Osceola,-81.335555,28.287159


In [27]:
## sales dimension table
sales_dim = property_record[['lastSalePrice', 'lastSaleDate']].drop_duplicates().reset_index(drop=True)
sales_dim.index.name = 'sales_id'

In [28]:
sales_dim.head()

,lastSalePrice,lastSaleDate
sales_id,,
0,325990.000000,2021-09-02 00:00:00+00:00
1,330000.000000,2014-10-03 00:00:00+00:00
2,239000.000000,2012-03-27 00:00:00+00:00
3,398627.340611,1900-01-01 00:00:00
4,217800.000000,1900-01-01 00:00:00


In [29]:
features_dim = property_record[['features', 'propertyType','zoning']].drop_duplicates().reset_index(drop=True)
features_dim.index.name= 'features_id'

In [30]:
features_dim.head()

,features,propertyType,zoning
features_id,,,
0,"{""exteriorType"": ""Stucco"", ""floorCount"": 1, ""g...",Single Family,PAD
1,"{""architectureType"": ""French Provincial"", ""coo...",Single Family,R5
2,"{""architectureType"": ""Condo / Apartment"", ""gar...",Condo,R-1:SINGLE FAM-RES
3,Unknown,Single Family,Unknown
4,"{""cooling"": true, ""coolingType"": ""Package"", ""e...",Single Family,OPUD


In [31]:
fact_table.to_csv(r"C:\Users\HP\Desktop\AMDARI\Projects\API_PostgresSQL ETL Pipeline for Zipco\data\property_fact_table.csv", index=False)
location_dim.to_csv(r"C:\Users\HP\Desktop\AMDARI\Projects\API_PostgresSQL ETL Pipeline for Zipco\data\location_dimension.csv", index=False)
sales_dim.to_csv(r"C:\Users\HP\Desktop\AMDARI\Projects\API_PostgresSQL ETL Pipeline for Zipco\data\sales_dimension.csv", index= False)
features_dim.to_csv(r"C:\Users\HP\Desktop\AMDARI\Projects\API_PostgresSQL ETL Pipeline for Zipco\data\features_dimension.csv", index= False)

# data loading

In [32]:
# define function to connect to pgadmin
def get_db_connection():
    connection = psycopg2.connect(
        host = 'localhost',
        database = 'postgres',
        user = 'postgres',
        password = 'Abdazeez@0312'
    
    )
    return connection
conn = get_db_connection()

In [56]:
# create tables with function
def create_tables():
    conn = get_db_connection()
    cursor = conn.cursor()
    create_table_query = '''CREATE SCHEMA IF NOT EXISTS zapbank;
    
                            DROP TABLE IF EXISTS zapbank.fact_table;
                            DROP TABLE IF EXISTS zapbank.location_dim;
                            DROP TABLE IF EXISTS zapbank.sales_dim;
                            DROP TABLE IF EXISTS zapbank.features_dim;
                            
                            CREATE TABLE zapbank.fact_table(
                                addressLine1 VARCHAR(255),
                                city VARCHAR(100),
                                state VARCHAR(50),
                                zipCode INTEGER,
                                formattedAddress VARCHAR(255),
                                squareFootage NUMERIC,
                                yearBuilt NUMERIC,
                                bathrooms NUMERIC,
                                bedrooms NUMERIC,
                                lotSize NUMERIC,
                                propertyType VARCHAR(255),
                                longitude NUMERIC,
                                latitude NUMERIC
                            );
                            
                            CREATE TABLE zapbank.location_dim(
                                location_id SERIAL PRIMARY KEY,
                                addressLine1 VARCHAR(255), 
                                city VARCHAR(100),
                                state VARCHAR(50), 
                                zipCode INTEGER, 
                                county VARCHAR(100), 
                                longitude NUMERIC,
                                latitude NUMERIC
                            );
                            
                            CREATE TABLE zapbank.sales_dim(
                                sales_id SERIAL PRIMARY KEY,
                                lastSalePrice NUMERIC, 
                                lastSaleDate DATE
                            );
                            
                            CREATE TABLE zapbank.features_dim(
                                features_id SERIAL PRIMARY KEY,
                                features JSONB,
                                propertyType TEXT,
                                zoning TEXT
                            ); '''
    
    
    cursor.execute(create_table_query)
    conn.commit()
    cursor.close()
    conn.close()
    
    
create_tables()
    

In [45]:
# create a function to load the csv data into the database
def load_data_from_csv_to_table(csv_path, table_name):
    conn = get_db_connection()
    cursor = conn.cursor()
    with open(csv_path,'r', encoding='utf-8') as file:
        reader = csv.reader(file)
        next(reader)  #skip the header row
        for row in reader:
            placeholders = ', '.join(['%s'] * len(row))
            query = f'INSERT INTO {table_name} VALUES ({placeholders});'
            cursor.execute(query, row)
    conn.commit()
    cursor.close()
    conn.close()
    
    # Function to convert data types based on column expectations


In [46]:
# fact table insertion
fact_csv_path = r"C:\Users\HP\Desktop\AMDARI\Projects\API_PostgresSQL ETL Pipeline for Zipco\data\property_fact_table.csv"
load_data_from_csv_to_table(fact_csv_path, 'zapbank.fact_table')


In [57]:
# location table insertion

location_csv_path = r"C:\Users\HP\Desktop\AMDARI\Projects\API_PostgresSQL ETL Pipeline for Zipco\data\location_dimension.csv"
load_data_from_csv_to_table(location_csv_path, 'zapbank.location_dim')


In [55]:
def load_data_from_csv_to_table(csv_path, table_name):
    conn = get_db_connection()
    cursor = conn.cursor()
    with open(csv_path, 'r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        columns = reader.fieldnames
        for row in reader:
            placeholders = ', '.join(['%s'] * len(columns))
            query = f'INSERT INTO {table_name} ({", ".join(columns)}) VALUES ({placeholders});'
            values = [row[col] for col in columns]
            cursor.execute(query, values)
    conn.commit()
    cursor.close()
    conn.close()


In [66]:

# Function to clean and validate JSON strings
def clean_json_string(json_str):
    if not json_str or json_str.strip() == '':  # Check for empty strings
        return json.dumps({"error": "Empty JSON"})
    
    try:
        # Try to load the string as JSON
        json_object = json.loads(json_str)
        return json.dumps(json_object)  # Convert to JSON string
    except (TypeError, json.JSONDecodeError):
        # Handle invalid JSON
        return json.dumps({"error": "Invalid JSON"})  # Or return some default value

# Apply cleaning function
features_dim['features'] = features_dim['features'].apply(clean_json_string)

# Save the cleaned DataFrame to a new CSV
cleaned_feature_csv_path = r"C:\Users\HP\Desktop\AMDARI\Projects\API_PostgresSQL ETL Pipeline for Zipco\data\cleaned_features_dimension.csv"
features_dim.to_csv(cleaned_feature_csv_path, index=False)


In [67]:
def load_data_from_csv_to_feature_table(csv_path, table_name):
    conn = get_db_connection()
    cursor = conn.cursor()
    
    with open(csv_path, 'r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        columns = reader.fieldnames
        
        for row in reader:
            values = []
            for col in columns:
                value = row[col]
                if col == 'features':
                    value = json.dumps(json.loads(value))
                values.append(value)
            
            placeholders = ', '.join(['%s'] * len(columns))
            query = f'INSERT INTO {table_name} ({", ".join(columns)}) VALUES ({placeholders});'
            cursor.execute(query, values)
    
    conn.commit()
    cursor.close()
    conn.close()


In [68]:
# Feature table insertion
#Use the cleaned CSV
load_data_from_csv_to_feature_table(cleaned_feature_csv_path, 'zapbank.features_dim')

In [73]:
# sales table insertion
sales_csv_path =r"C:\Users\HP\Desktop\AMDARI\Projects\API_PostgresSQL ETL Pipeline for Zipco\data\sales_dimension.csv"
load_data_from_csv_to_table(sales_csv_path, 'zapbank.sales_dim')


In [ ]:

print("All the Data has been loaded successfully into their respective schema and tables")